In [166]:
import os
import json
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns

# Cleaning Data

## performance & fund_detail table

In [167]:
dir = '../data/raw/20231212/funds/'
symbol_list = []
category = []
asset_class = []
inception_date = []
min_inv = []
exp_ratio = []
return_ytd = []
return_1_year = []
return_3_year = []
return_5_year = []
return_10_year = []
return_inception = []
return_bench_ytd = []
return_bench_1_year = []
return_bench_3_year = []
return_bench_5_year = []
return_bench_10_year = []
return_bench_inception = []
risk_level = []
product_summary = []

for filename in os.listdir(dir):
    file_path = os.path.join(dir,filename)
    with open(file_path,'r') as f:
        fund_dict = json.load(f)
    pattern = r'\/([A-Z]+)\.json'
    match = re.search(pattern, file_path)
    symbol = match.group(1)
    symbol_list.append(symbol)
    
    category.append(fund_dict['key_fact_table']['Category'])
    asset_class.append(fund_dict['key_fact_table']['Asset class'])

    if 'product_summary' in fund_dict:
        product_summary.append(fund_dict['product_summary'])
    else:
        product_summary.append("NA")

    if 'Inception date' in fund_dict['key_fact_table']:
        inception_date.append(fund_dict['key_fact_table']['Inception date'])
    else:
        inception_date.append(None)

    if 'risk_level' in fund_dict:
        risk_level.append(fund_dict['risk_level'])
    else:
        risk_level.append(None)

    if 'perf_table' in fund_dict:
        perf_table = fund_dict['perf_table']

        if symbol in perf_table:
            symbol_key = symbol
        elif symbol+'1' in perf_table:
            symbol_key = symbol + '1'
        elif symbol+'2' in perf_table:
            symbol_key = symbol+'2'
        elif symbol+'4' in perf_table:
            symbol_key = symbol+'4'
        else:
            symbol_key = symbol+' (Market price)'
        
        if 'Benchmark1' in perf_table:
            benchmark_key = 'Benchmark1'
        elif 'Benchmark3' in perf_table:
            benchmark_key = 'Benchmark3'
        else:
            benchmark_key = 'Benchmark'

        for row in zip(perf_table['index'],perf_table[symbol_key],perf_table[benchmark_key]):
            if row[0]=="YTD":
                return_ytd.append(row[1])
                return_bench_ytd.append(row[2])
            if row[0]=="1-yr":
                return_1_year.append(row[1])
                return_bench_1_year.append(row[2])
            if row[0]=="3-yr":
                return_3_year.append(row[1])
                return_bench_3_year.append(row[2])
            if row[0]=="5-yr":
                return_5_year.append(row[1])
                return_bench_5_year.append(row[2])
            if row[0]=="10-yr":
                return_10_year.append(row[1])
                return_bench_10_year.append(row[2])
            if row[0]=="Since inception":
                return_inception.append(row[1]) 
                return_bench_inception.append(row[2])
    else:
        return_ytd.append(None)
        return_bench_ytd.append(None)
        return_1_year.append(None)
        return_bench_1_year.append(None)
        return_3_year.append(None)
        return_bench_3_year.append(None)
        return_5_year.append(None)
        return_bench_5_year.append(None)
        return_10_year.append(None)
        return_bench_10_year.append(None)
        return_inception.append(None)
        return_bench_inception.append(None)

    if 'min_investment' in fund_dict:
        min_inv.append(fund_dict['min_investment'])
    else:
        min_inv.append(None)
    
    if 'exp_ratio' in fund_dict:
        exp_ratio.append(fund_dict['exp_ratio'])
    else:
        exp_ratio.append(None)    

In [169]:
table_col = ['symbol','category','product_summary','asset_class','inception_date','minimum_investment','expense_ratio','fund_return_ytd','average_annual_fund_return_for_1_year','average_annual_fund_return_for_3_year','average_annual_fund_return_for_5_year','average_annual_fund_return_for_10_year','average_annual_fund_return_since_inception','benchmark_return_ytd','average_annual_benchmark_return_for_1_year','average_annual_benchmark_return_for_3_year','average_annual_benchmark_return_for_5_year','average_annual_benchmark_return_for_10_year','average_annual_benchmark_return_since_inception','risk_level']

fund_df = pd.DataFrame(columns=table_col,data=list(zip(symbol_list,category,product_summary,asset_class,inception_date,min_inv,exp_ratio,return_ytd,return_1_year,return_3_year,return_5_year,return_10_year,return_inception,return_bench_ytd,return_bench_1_year,return_bench_3_year,return_bench_5_year,return_bench_10_year,return_bench_inception,risk_level)))

In [170]:
fund_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 20 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   symbol                                           350 non-null    object
 1   category                                         350 non-null    object
 2   product_summary                                  350 non-null    object
 3   asset_class                                      350 non-null    object
 4   inception_date                                   350 non-null    object
 5   minimum_investment                               345 non-null    object
 6   expense_ratio                                    350 non-null    object
 7   fund_return_ytd                                  350 non-null    object
 8   average_annual_fund_return_for_1_year            350 non-null    object
 9   average_annual_fund_return_for_3_year      

In [171]:
def perc_to_decimal(string):
    if string:
        return round(float((string.strip().replace('%','')))/100,5)
    else:
        return string

In [172]:
def clean_risk_level(string):
    risk_level = int(string.strip())
    if risk_level == 1:
        return 'Conservative'
    elif risk_level == 2:
        return 'Conservative to Moderate'
    elif risk_level == 3:
        return 'Moderate'
    elif risk_level == 4:
        return 'Moderate to Aggressive'
    else:
        return 'Aggressive'

In [173]:
def clean_min_investment(string):
    return int(float(string.replace(',','').replace('$','')))

In [174]:
# Remove funds with null min_inv --> Closed not allowed to be bought by investor anymore
cleaned_df = fund_df.copy()
cleaned_df = cleaned_df[cleaned_df['minimum_investment'].notna()]

# replace unicode dash with Null value
cleaned_df = cleaned_df.replace('\u2014',None)

# clean risk level
cleaned_df['risk_level'] = cleaned_df['risk_level'].apply(clean_risk_level)

# convert percentages string to float
percentage_cols = ['expense_ratio','fund_return_ytd','average_annual_fund_return_for_1_year','average_annual_fund_return_for_3_year','average_annual_fund_return_for_5_year','average_annual_fund_return_for_10_year','average_annual_fund_return_since_inception','benchmark_return_ytd','average_annual_benchmark_return_for_1_year','average_annual_benchmark_return_for_3_year','average_annual_benchmark_return_for_5_year','average_annual_benchmark_return_for_10_year','average_annual_benchmark_return_since_inception']
for col in percentage_cols:
    print(col)
    cleaned_df[col] = cleaned_df[col].apply(perc_to_decimal)

cleaned_df['inception_date'] = pd.to_datetime(cleaned_df['inception_date'])

cleaned_df['minimum_investment'] = cleaned_df['minimum_investment'].apply(clean_min_investment)

expense_ratio
fund_return_ytd
average_annual_fund_return_for_1_year
average_annual_fund_return_for_3_year
average_annual_fund_return_for_5_year
average_annual_fund_return_for_10_year
average_annual_fund_return_since_inception
benchmark_return_ytd
average_annual_benchmark_return_for_1_year
average_annual_benchmark_return_for_3_year
average_annual_benchmark_return_for_5_year
average_annual_benchmark_return_for_10_year
average_annual_benchmark_return_since_inception


In [175]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 345 entries, 0 to 349
Data columns (total 20 columns):
 #   Column                                           Non-Null Count  Dtype         
---  ------                                           --------------  -----         
 0   symbol                                           345 non-null    object        
 1   category                                         345 non-null    object        
 2   product_summary                                  345 non-null    object        
 3   asset_class                                      345 non-null    object        
 4   inception_date                                   345 non-null    datetime64[ns]
 5   minimum_investment                               345 non-null    int64         
 6   expense_ratio                                    345 non-null    float64       
 7   fund_return_ytd                                  341 non-null    float64       
 8   average_annual_fund_return_for_1_year        

In [96]:
perf_df = cleaned_df[['symbol','fund_return_ytd','average_annual_fund_return_for_1_year','average_annual_fund_return_for_3_year','average_annual_fund_return_for_5_year','average_annual_fund_return_for_10_year','average_annual_fund_return_since_inception','benchmark_return_ytd','average_annual_benchmark_return_for_1_year','average_annual_benchmark_return_for_3_year','average_annual_benchmark_return_for_5_year','average_annual_benchmark_return_for_10_year','average_annual_benchmark_return_since_inception']].copy()
fund_detail_df = cleaned_df[['symbol','category','product_summary','asset_class','inception_date','minimum_investment','expense_ratio','risk_level']].copy()
perf_df.to_csv('../data/processed/20231212/perf.csv')
fund_detail_df.to_csv('../data/processed/20231212/fund_detail.csv')

In [149]:
dir = '../data/raw/20231212/funds/'
sec_symbol_list = []
sec_list = []
sec_alloc = []
reg_symbol_list = []
reg_list = []
reg_alloc = []


for filename in os.listdir(dir):
    file_path = os.path.join(dir,filename)
    with open(file_path,'r') as f:
        fund_dict = json.load(f)
    pattern = r'\/([A-Z]+)\.json'
    match = re.search(pattern, file_path)
    symbol = match.group(1)
    symbol_list.append(symbol)
    
    if 'weighted_exposure_table' in fund_dict:
        if ' Sectors' in fund_dict['weighted_exposure_table']:
            for sec,alloc in zip(fund_dict['weighted_exposure_table'][' Sectors'][' Sectors'],fund_dict['weighted_exposure_table'][' Sectors'][symbol]):
                sec_symbol_list.append(symbol)
                sec_list.append(sec.strip())
                sec_alloc.append(alloc.strip())
                
        if ' Regions' in fund_dict['weighted_exposure_table']:
            for reg,alloc in zip(fund_dict['weighted_exposure_table'][' Regions'][' Regions'],fund_dict['weighted_exposure_table'][' Regions'][symbol]):
                reg_symbol_list.append(symbol)
                reg_list.append(reg.strip())
                reg_alloc.append(alloc.strip())
        else:
            reg_symbol_list.append(symbol)
            reg_list.append('North America')
            reg_alloc.append("100%")
        


In [150]:
sector_df = pd.DataFrame({'symbol':sec_symbol_list,'sector':sec_list,'allocation':sec_alloc})
region_df = pd.DataFrame({'symbol':reg_symbol_list,'region':reg_list,'allocation':reg_alloc})

In [151]:
sector_df['allocation'] = sector_df['allocation'].apply(perc_to_decimal)
region_df['allocation'] = region_df['allocation'].apply(perc_to_decimal)

valid_symbols = cleaned_df['symbol'].tolist()

sector_df = sector_df[sector_df['symbol'].isin(valid_symbols)]
region_df = region_df[region_df['symbol'].isin(valid_symbols)]

In [152]:
sector_df.to_csv('../data/processed/20231212/sector.csv')
region_df.to_csv('../data/processed/20231212/region.csv')

In [163]:
no_sector_symbol = list(set(cleaned_df.symbol.tolist())-set(sector_df.symbol.tolist()))

In [164]:
cleaned_df[cleaned_df['symbol'].isin(no_sector_symbol)]

,symbol,category,asset_class,inception_date,minimum_investment,expense_ratio,fund_return_ytd,average_annual_fund_return_for_1_year,average_annual_fund_return_for_3_year,average_annual_fund_return_for_5_year,average_annual_fund_return_for_10_year,average_annual_fund_return_since_inception,benchmark_return_ytd,average_annual_benchmark_return_for_1_year,average_annual_benchmark_return_for_3_year,average_annual_benchmark_return_for_5_year,average_annual_benchmark_return_for_10_year,average_annual_benchmark_return_since_inception,risk_level
0,VTIVX,Target-Date 2041-2045,Balanced,2003-10-27,1000,0.0008,0.1385,0.0935,0.0453,0.0821,0.0759,0.0775,0.1404,0.1021,0.0483,0.0865,0.0792,0.0797,Moderate to Aggressive
2,VCORX,Intermediate-Term Bond,Intermediate-Term Bond,2016-03-28,3000,0.0020,0.0198,0.0151,-0.0439,0.0135,NaN,0.0105,0.0178,0.0132,-0.0446,0.0079,0.0140,0.0075,Conservative to Moderate
4,VWILX,Foreign Large Growth,International/Global Stock,2001-08-13,50000,0.0034,0.0958,0.0417,-0.0711,0.0804,0.0684,0.0726,0.1009,0.0926,0.0167,0.0506,0.0341,0.0464,Aggressive
8,VMLTX,Muni National Short,Short-Term Bond,1987-08-31,3000,0.0017,0.0291,0.0318,0.0018,0.0159,0.0139,0.0362,0.0264,0.0286,-0.0014,0.0143,0.0133,NaN,Conservative
11,VGLT,Long Government,Long-Term Bond,2009-11-19,1,0.0004,-0.0467,-0.0678,-0.1415,-0.0189,0.0117,0.0248,-0.0511,-0.0672,-0.1416,-0.0181,0.0123,0.0255,Moderate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,VTTVX,Target-Date 2021-2025,Balanced,2003-10-27,1000,0.0008,0.0977,0.0647,0.0159,0.0577,0.0579,0.0633,0.0996,0.0722,0.0194,0.0622,0.0612,0.0655,Moderate
345,VTPSX,Foreign Large Blend,International/Global Stock,2010-11-30,100000000,0.0007,0.0995,0.0756,0.0204,0.0527,0.0375,0.0463,0.1009,0.0952,0.0201,0.0538,0.0378,0.0463,Aggressive
346,VBTLX,Intermediate-Term Bond,Intermediate-Term Bond,2001-11-12,3000,0.0005,0.0193,0.0132,-0.0447,0.0074,0.0135,0.0314,0.0178,0.0132,-0.0446,0.0079,0.0140,0.0327,Conservative to Moderate
347,VBPIX,World Stock,International/Global Stock,2017-12-14,3000,0.0059,0.0801,0.0279,-0.0499,0.1359,NaN,0.1271,0.1660,0.1201,0.0569,0.0907,0.0760,0.0737,Moderate to Aggressive
